In [251]:
import psycopg2
import pandas
import pandas.io.sql as psql
import ConfigParser
import os

#USER_CRED_FILE = os.path.join(os.path.expanduser('~'), '.ipynb_dslab_sandbox.cred')
USER_CRED_FILE = os.path.join(os.path.expanduser('~'), '.ipynb_dslab_olddca.cred')

def fetchDBCredentials(dbcred_file=USER_CRED_FILE):
    """
       Read database access credentials from the file in $HOME/.ipynb_dslab.cred
    """
    #Read database credentials from user supplied file
    conf = ConfigParser.ConfigParser()
    conf.read(dbcred_file)
    #host, port, user, database, password
    host = conf.get('database_creds','host')
    port = conf.get('database_creds','port')
    user = conf.get('database_creds','user')
    database = conf.get('database_creds','database')
    password = conf.get('database_creds','password')

    #Initialize connection string
    conn_str =  """dbname='{database}' user='{user}' host='{host}' port='{port}' password='{password}'""".format(                       
                    database=database,
                    host=host,
                    port=port,
                    user=user,
                    password=password
            )
    return conn_str

# Steps to create csv file using hadoop

1. convert m4a files to mp3 using VLC media player
2. transfer the mp3 files to radhar5@10.68.128.150 hadoop master
3. ssh radhar5@10.68.128.150
4. hadoop fs -ls /user/radhar5/audio
5. hadoop fs -copyFromLocal *.mp3 /user/radhar5/audio
6. create audio_file_list.txt
7. hadoop fs -copyFromLocal audio_file_list_testrec.txt /user/radhar5/audio/
8. hadoop jar convert_audio_to_seqfile_hadoop.jar ConvertAudioToSequenceFileHadoop "/user/radhar5/audio/audio_file_list_testrec.txt" "/user/radhar5/audio/audio_SequenceFile_testrec"
9. hadoop fs -copyToLocal /user/radhar5/audio/audio_SequenceFile_testrec/part-r-00000 audio_seq_file_testrec.csv

In [252]:
conn = psycopg2.connect(fetchDBCredentials())
df = psql.read_sql("""select random() as x, random() as y from generate_series(1, 10) q;""", conn)
df.head()

,x,y
0,0.850318,0.405844
1,0.058902,0.560908
2,0.919126,0.150197
3,0.957200,0.096068
4,0.467490,0.579158


In [181]:
%matplotlib inline
import numpy as np
import urllib2
import StringIO
from matplotlib import pyplot as plt
from array import array

In [140]:
sql = """create schema audiofp""";
psql.execute(sql,conn)
conn.commit()

# gpfdist -d /home/radhar5/audio_seq -p 8089 -V -m 255000000

In [182]:
#LOCATTION 'gpfdist://localhost:8089 when using sandbox
sql = """drop external table if exists audiofp.training_audio_ext;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create external table audiofp.training_audio_ext (
        aud_name varchar,
        audio varchar
    )
    LOCATION('gpfdist://mdw:8089/audio_seq_file_vlccmd.csv')
    FORMAT 'TEXT' (DELIMITER AS E'\t');
"""
psql.execute(sql,conn)
conn.commit()

In [183]:
sql = """select count(*) from audiofp.training_audio_ext"""
psql.read_sql(sql,conn)

,count
0,52


In [184]:
# Create the training internal table
sql = """drop table if exists audiofp.training_audio;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table audiofp.training_audio AS
        (select * from audiofp.training_audio_ext)
    distributed by (aud_name)
"""
psql.execute(sql,conn)
conn.commit()

In [185]:
sql= """select aud_name from audiofp.training_audio order by 1"""
psql.read_sql(sql,conn)

,aud_name
0,/user/radhar5/audio/Agnes-_Release_Me.mp3vlc_c...
1,/user/radhar5/audio/A-Ha-_Foot_Of_The_Mountain...
2,/user/radhar5/audio/Alexander_Rybak-_Fairytale...
3,/user/radhar5/audio/Amy_MacDonald-_This_Is_The...
4,/user/radhar5/audio/Beyonce-_Halo.mp3vlc_cmd.mp3
5,/user/radhar5/audio/Beyonce-_Sweet_Dreams.mp3v...
6,/user/radhar5/audio/Black_Eyed_Peas-_Boom_Boom...
7,/user/radhar5/audio/Black_Eyed_Peas-_I_Gotta_F...
8,/user/radhar5/audio/Cascada-_Evacuate_The_Danc...
9,/user/radhar5/audio/Cassandra_Steen-_Stadt.mp3...


In [186]:
sql = """select count(*) from audiofp.training_audio"""
psql.read_sql(sql,conn)
# Should ouput 907 images

,count
0,52


In [220]:
import os
import fnmatch
import numpy as np
from pydub import AudioSegment
from pydub.utils import audioop
#import wavio
from hashlib import sha1
#import wave as _wave
import numpy as _np

import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import (generate_binary_structure,
                                      iterate_structure, binary_erosion)
import hashlib
from operator import itemgetter

IDX_FREQ_I = 0
IDX_TIME_J = 1

######################################################################
# Sampling rate, related to the Nyquist conditions, which affects
# the range frequencies we can detect.
DEFAULT_FS = 44100

######################################################################
# Size of the FFT window, affects frequency granularity
DEFAULT_WINDOW_SIZE = 4096

######################################################################
# Ratio by which each sequential window overlaps the last and the
# next window. Higher overlap will allow a higher granularity of offset
# matching, but potentially more fingerprints.
DEFAULT_OVERLAP_RATIO = 0.5

######################################################################
# Degree to which a fingerprint can be paired with its neighbors --
# higher will cause more fingerprints, but potentially better accuracy.
DEFAULT_FAN_VALUE = 15

######################################################################
# Minimum amplitude in spectrogram in order to be considered a peak.
# This can be raised to reduce number of fingerprints, but can negatively
# affect accuracy.
DEFAULT_AMP_MIN = 10

######################################################################
# Number of cells around an amplitude peak in the spectrogram in order
# for Dejavu to consider it a spectral peak. Higher values mean less
# fingerprints and faster matching, but can potentially affect accuracy.
PEAK_NEIGHBORHOOD_SIZE = 20

######################################################################
# Thresholds on how close or far fingerprints can be in time in order
# to be paired as a fingerprint. If your max is too low, higher values of
# DEFAULT_FAN_VALUE may not perform as expected.
MIN_HASH_TIME_DELTA = 0
MAX_HASH_TIME_DELTA = 200

######################################################################
# If True, will sort peaks temporally for fingerprinting;
# not sorting will cut down number of fingerprints, but potentially
# affect performance.
PEAK_SORT = True

######################################################################
# Number of bits to throw away from the front of the SHA1 hash in the
# fingerprint calculation. The more you throw away, the less storage, but
# potentially higher collisions and misclassifications when identifying songs.
FINGERPRINT_REDUCTION = 20


def extract_fing_from_buffer(audiobitstream,limit=None,song_name=None):
	def _wav2array(nchannels, sampwidth, data):
		num_samples, remainder = divmod(len(data), sampwidth * nchannels)
		if remainder > 0:
			raise ValueError('The length of data is not a multiple of '
			                 'sampwidth * num_channels.')
		if sampwidth > 4:
			raise ValueError("sampwidth must not be greater than 4.")
		if sampwidth == 3:
			a = _np.empty((num_samples, nchannels, 4), dtype=_np.uint8)
			raw_bytes = _np.fromstring(data, dtype=_np.uint8)
			a[:, :, :sampwidth] = raw_bytes.reshape(-1, nchannels, sampwidth)
			a[:, :, sampwidth:] = (a[:, :, sampwidth - 1:sampwidth] >> 7) * 255
			result = a.view('<i4').reshape(a.shape[:-1])
		else:
			# 8 bit samples are stored as unsigned ints; others as signed ints.
			dt_char = 'u' if sampwidth == 1 else 'i'
			a = _np.fromstring(data, dtype='<%s%d' % (dt_char, sampwidth))
			result = a.reshape(-1, nchannels)
		return result
	def readwav(file):
		wav = _wave.open(file)
		rate = wav.getframerate()
		nchannels = wav.getnchannels()
		sampwidth = wav.getsampwidth()
		nframes = wav.getnframes()
		data = wav.readframes(nframes)
		wav.close()
		array = _wav2array(nchannels, sampwidth, data)
		return rate, sampwidth, array
	def read(fileContent, limit=None):
	# pydub does not support 24-bit wav files, use wavio when this occurs
		#try:
			#audiofile = AudioSegment.from_file(filename)
			#filehandle =  open(filename, mode='rb')
			#fileContent = filehandle.read()
		print("read audio bitstream")
		audiofile = AudioSegment.from_buffer(fileContent)
		if limit:
			audiofile = audiofile[:limit * 1000]
		data = np.fromstring(audiofile._data, np.int16)
		print(data[44100:44120])
		print("after reading audio bitstream")
		print(audiofile.frame_rate)
		print(audiofile.channels)
		channels = []
		for chn in xrange(audiofile.channels):
			channels.append(data[chn::audiofile.channels])
		print(audiofile.frame_rate)
		fs = audiofile.frame_rate
		#except audioop.error:
		#	fs, _, audiofile = readwav(filename)
		#	if limit:
		#		audiofile = audiofile[:limit * 1000]
		#	audiofile = audiofile.T
		#	audiofile = audiofile.astype(np.int16)
		#	channels = []
		#	for chn in audiofile:
		#		channels.append(chn)
		return channels, audiofile.frame_rate    
	def get_2D_peaks(arr2D, plot=False, amp_min=DEFAULT_AMP_MIN):
		# http://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.morphology.iterate_structure.html#scipy.ndimage.morphology.iterate_structure
		struct = generate_binary_structure(2, 1)
		neighborhood = iterate_structure(struct, PEAK_NEIGHBORHOOD_SIZE)
		# find local maxima using our fliter shape
		local_max = maximum_filter(arr2D, footprint=neighborhood) == arr2D
		background = (arr2D == 0)
		eroded_background = binary_erosion(background, structure=neighborhood,
		                                   border_value=1)
		# Boolean mask of arr2D with True at peaks
		detected_peaks = local_max - eroded_background
		# extract peaks
		amps = arr2D[detected_peaks]
		j, i = np.where(detected_peaks)
		# filter peaks
		amps = amps.flatten()
		peaks = zip(i, j, amps)
		peaks_filtered = [x for x in peaks if x[2] > amp_min]  # freq, time, amp
		# get indices for frequency and time
		frequency_idx = [x[1] for x in peaks_filtered]
		time_idx = [x[0] for x in peaks_filtered]
		if plot:
			# scatter of the peaks
			fig, ax = plt.subplots()
			ax.imshow(arr2D)
			ax.scatter(time_idx, frequency_idx)
			ax.set_xlabel('Time')
			ax.set_ylabel('Frequency')
			ax.set_title("Spectrogram")
			plt.gca().invert_yaxis()
			plt.show()
		return zip(frequency_idx, time_idx)
	def generate_hashes(peaks, fan_value=DEFAULT_FAN_VALUE):
		"""
		Hash list structure:
		   sha1_hash[0:20]    time_offset
		[(e05b341a9b77a51fd26, 32), ... ]
		"""
		if PEAK_SORT:
			peaks.sort(key=itemgetter(1))
		for i in range(len(peaks)):
			for j in range(1, fan_value):
				if (i + j) < len(peaks):
					freq1 = peaks[i][IDX_FREQ_I]
					freq2 = peaks[i + j][IDX_FREQ_I]
					t1 = peaks[i][IDX_TIME_J]
					t2 = peaks[i + j][IDX_TIME_J]
					t_delta = t2 - t1
					if t_delta >= MIN_HASH_TIME_DELTA and t_delta <= MAX_HASH_TIME_DELTA:
						h = hashlib.sha1(
						    "%s|%s|%s" % (str(freq1), str(freq2), str(t_delta)))
						yield (h.hexdigest()[0:FINGERPRINT_REDUCTION], t1)
	def fingerprint(channel_samples, Fs=DEFAULT_FS,
					wsize=DEFAULT_WINDOW_SIZE,
					wratio=DEFAULT_OVERLAP_RATIO,
					fan_value=DEFAULT_FAN_VALUE,
					amp_min=DEFAULT_AMP_MIN):
		"""
		FFT the channel, log transform output, find local maxima, then return
		locally sensitive hashes.
		"""
		# FFT the signal and extract frequency components
		arr2D = mlab.specgram(
				channel_samples,
				NFFT=wsize,
				Fs=Fs,
				window=mlab.window_hanning,
				noverlap=int(wsize * wratio))[0]
		# apply log transform since specgram() returns linear array
		arr2D = 10 * np.log10(arr2D)
		arr2D[arr2D == -np.inf] = 0  # replace infs with zeros
		# find local maxima
		local_maxima = get_2D_peaks(arr2D, plot=False, amp_min=amp_min)
		# return hashes
		return generate_hashes(local_maxima, fan_value=fan_value)
	channels, Fs = read(audiobitstream, limit)
	result = set()
	channel_amount = len(channels)
	for channeln, channel in enumerate(channels):
		# TODO: Remove prints or change them into optional logging.
		print("Fingerprinting channel %d/%d " % (channeln + 1,
		                                               channel_amount
		                                               ))
		hashes = fingerprint(channel, Fs=Fs)
		print("Finished channel %d/%d " % (channeln + 1, channel_amount
		                                         ))
		result |= set(hashes)
	import pandas as pd
	song_names = []
	time_offsets = np.zeros(len(result),dtype=np.float)
	fing = []
	i = 0
	for hash_v in result:
		song_names.append(song_name)
		fing.append(hash_v[0])
		time_offsets[i] = hash_v[1]
		i=i+1
	return(pd.DataFrame(zip(song_names,time_offsets,fing)))
	

In [221]:
from tempfile import TemporaryFile, NamedTemporaryFile
# First let us compute the SIFT features using the Python binding for the OpenCV 
# library locally in this notebook to ensure our images have been stored properly
sql = """
        SELECT
            aud_name,
            audio
        FROM
            audiofp.training_audio
        WHERE aud_name like '%/user/radhar5/audio/A-Ha%'
        LIMIT 1;        
"""

df = psql.read_sql(sql,conn)

imgname = np.array(df.loc[0]['aud_name'])

aud = df.loc[0]['audio']
buf = aud.split(',')
buf = map(int, buf)
buf = array('b',buf)
buf = buf.tostring()

input_file = NamedTemporaryFile(mode='wb', delete=False)
input_file.write(buf)
input_file.flush()

tempfilename = open(input_file.name,mode="rb")

#aud_arr = np.asarray(bytearray(buf), dtype=np.uint8)

aud_arr = tempfilename.read()
print(aud_arr[0:10])
print(type(aud_arr))
AudioSegment.converter = "/Applications/ffmpeg"
extract_fing_from_buffer(aud_arr,limit=None,song_name='test_name')



��pd �  i
<type 'str'>
read audio bitstream
[ 2719 -4826  2126 -5830  1571 -6716  1078 -7419   658 -7885   303 -8090
    25 -8004  -162 -7614  -263 -6949  -281 -6050]
after reading audio bitstream
44100
2
44100
Fingerprinting channel 1/2 
Finished channel 1/2 
Fingerprinting channel 2/2 
Finished channel 2/2 


,0,1,2
0,test_name,3590,82d325b7f04cc06c467e
1,test_name,4656,c93205d6d059171a950b
2,test_name,3853,164f1c9adf376ba048d7
3,test_name,3118,0e10257831f6c0285e9d
4,test_name,3279,9af359c0a8016aa6db0a
5,test_name,2989,eae87a181c70a7405a98
6,test_name,3520,d8ffc6b4145ff01c8bf9
7,test_name,2840,baf4dbee0228b83c42ae
8,test_name,1233,3688874fd5a79ef1a41e
9,test_name,2978,13805dcd7e4e25fdc011


In [152]:
sql =  """drop type if exists fingerprint_result cascade;"""
psql.execute(sql,conn)
conn.commit()

sql = """
create type fingerprint_result
as
(
	time_offset float, 
	fing_val text
	
);
"""
psql.execute(sql,conn)
conn.commit()

In [206]:
# We will first create the UDF
sql = """drop function if exists audiofp.fing_extract_plpy(audio_bit varchar) cascade;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create or replace function audiofp.fing_extract_plpy(audio_bit varchar) 
    returns setof fingerprint_result
    as 
    $$
        
        import os
        import fnmatch
        import numpy as np
        #import mlab
        #from pydub import AudioSegment
        #from pydub.utils import audioop
        #import wavio
        from hashlib import sha1
        #import wave as _wave
        import numpy as _np
        from array import array
        from tempfile import TemporaryFile, NamedTemporaryFile

        import numpy as np
        import matplotlib.mlab as mlab
        #import matplotlib.pyplot as plt
        from scipy.ndimage.filters import maximum_filter
        from scipy.ndimage.morphology import (generate_binary_structure,
                                              iterate_structure, binary_erosion)
        import hashlib
        from operator import itemgetter

        song_name = None
        limit = None
        
        IDX_FREQ_I = 0
        IDX_TIME_J = 1

       
        DEFAULT_FS = 44100

        
        DEFAULT_WINDOW_SIZE = 4096

       
        DEFAULT_OVERLAP_RATIO = 0.5

        
        DEFAULT_FAN_VALUE = 15

        
        DEFAULT_AMP_MIN = 10

       
        PEAK_NEIGHBORHOOD_SIZE = 20

        
        MIN_HASH_TIME_DELTA = 0
        MAX_HASH_TIME_DELTA = 200

       
        PEAK_SORT = True

        FINGERPRINT_REDUCTION = 20

        def _wav2array(nchannels, sampwidth, data):
            num_samples, remainder = divmod(len(data), sampwidth * nchannels)
            if remainder > 0:
                raise ValueError('The length of data is not a multiple of '
                                 'sampwidth * num_channels.')
            if sampwidth > 4:
                raise ValueError("sampwidth must not be greater than 4.")
            if sampwidth == 3:
                a = _np.empty((num_samples, nchannels, 4), dtype=_np.uint8)
                raw_bytes = _np.fromstring(data, dtype=_np.uint8)
                a[:, :, :sampwidth] = raw_bytes.reshape(-1, nchannels, sampwidth)
                a[:, :, sampwidth:] = (a[:, :, sampwidth - 1:sampwidth] >> 7) * 255
                result = a.view('<i4').reshape(a.shape[:-1])
            else:
                # 8 bit samples are stored as unsigned ints; others as signed ints.
                dt_char = 'u' if sampwidth == 1 else 'i'
                a = _np.fromstring(data, dtype='<%s%d' % (dt_char, sampwidth))
                result = a.reshape(-1, nchannels)
            return result
        def readwav(file):
            wav = _wave.open(file)
            rate = wav.getframerate()
            nchannels = wav.getnchannels()
            sampwidth = wav.getsampwidth()
            nframes = wav.getnframes()
            data = wav.readframes(nframes)
            wav.close()
            array = _wav2array(nchannels, sampwidth, data)
            return rate, sampwidth, array
        def read(fileContent, limit=None):
        # pydub does not support 24-bit wav files, use wavio when this occurs
            #try:
                #audiofile = AudioSegment.from_file(filename)
                #filehandle =  open(filename, mode='rb')
                #fileContent = filehandle.read()
            print("read audio bitstream")
            audiofile = AudioSegment.from_buffer(fileContent)
            if limit:
                audiofile = audiofile[:limit * 1000]
            data = np.fromstring(audiofile._data, np.int16)
            print("after reading audio bitstream")
            print(audiofile.frame_rate)
            print(audiofile.channels)
            channels = []
            for chn in xrange(audiofile.channels):
                channels.append(data[chn::audiofile.channels])
            print(audiofile.frame_rate)
            fs = audiofile.frame_rate
            #except audioop.error:
            #	fs, _, audiofile = readwav(filename)
            #	if limit:
            #		audiofile = audiofile[:limit * 1000]
            #	audiofile = audiofile.T
            #	audiofile = audiofile.astype(np.int16)
            #	channels = []
            #	for chn in audiofile:
            #		channels.append(chn)
            return channels, audiofile.frame_rate
        def read_frombuffer_so(fileContent):
            import ctypes
            from numpy.ctypeslib import ndpointer
            libfile ='/usr/local/greenplum-db/lib/decoding_encoding_1.so'
            dl = ctypes.cdll.LoadLibrary(libfile)
            decode_obj = dl.audio_decode_from_buffer_ret_samplerate_numchan
            decode_obj.restype = ndpointer(dtype=ctypes.c_int32,shape=(3,))
            plpy.info("casting to c_ubyte")
            plpy.info("before parsing")
            #plpy.info(fileContent[0:10])
            plpy.info(len(fileContent))
            ret_buf = decode_obj(ctypes.c_char_p(fileContent),ctypes.c_int32(0),ctypes.c_int32(len(fileContent)))
            num_seconds_to_decode = ret_buf[2]/(ret_buf[1]*ret_buf[0])
            #plpy.info(ret_buf)
            plpy.info("calling actual decode")
            decode_obj1 = dl.audio_decode_from_buffer_ret
            decode_obj1.restype = ndpointer(dtype=ctypes.c_int16,shape=((num_seconds_to_decode+1)*ret_buf[0]*ret_buf[1],))
            ret_buf1 = decode_obj1(ctypes.c_char_p(fileContent),ctypes.c_int32(0),ctypes.c_int32(len(fileContent)),ctypes.c_int32(num_seconds_to_decode))
            plpy.info("fin actual decode")
            num_chans = ret_buf[0]
            channels = []
            for chn in xrange(num_chans):
                channels.append(ret_buf1[chn::num_chans])
            framerate = ret_buf[1]
            return channels,framerate 
        def get_2D_peaks(arr2D, plot=False, amp_min=DEFAULT_AMP_MIN):
            # http://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.morphology.iterate_structure.html#scipy.ndimage.morphology.iterate_structure
            struct = generate_binary_structure(2, 1)
            neighborhood = iterate_structure(struct, PEAK_NEIGHBORHOOD_SIZE)
            # find local maxima using our fliter shape
            local_max = maximum_filter(arr2D, footprint=neighborhood) == arr2D
            background = (arr2D == 0)
            eroded_background = binary_erosion(background, structure=neighborhood,
                                               border_value=1)
            # Boolean mask of arr2D with True at peaks
            detected_peaks = local_max - eroded_background
            # extract peaks
            amps = arr2D[detected_peaks]
            j, i = np.where(detected_peaks)
            # filter peaks
            amps = amps.flatten()
            peaks = zip(i, j, amps)
            peaks_filtered = [x for x in peaks if x[2] > amp_min]  # freq, time, amp
            # get indices for frequency and time
            frequency_idx = [x[1] for x in peaks_filtered]
            time_idx = [x[0] for x in peaks_filtered]
            if plot:
                # scatter of the peaks
                fig, ax = plt.subplots()
                ax.imshow(arr2D)
                ax.scatter(time_idx, frequency_idx)
                ax.set_xlabel('Time')
                ax.set_ylabel('Frequency')
                ax.set_title("Spectrogram")
                plt.gca().invert_yaxis()
                plt.show()
            return zip(frequency_idx, time_idx)
        def generate_hashes(peaks, fan_value=DEFAULT_FAN_VALUE):
            if PEAK_SORT:
                peaks.sort(key=itemgetter(1))
            for i in range(len(peaks)):
                for j in range(1, fan_value):
                    if (i + j) < len(peaks):
                        freq1 = peaks[i][IDX_FREQ_I]
                        freq2 = peaks[i + j][IDX_FREQ_I]
                        t1 = peaks[i][IDX_TIME_J]
                        t2 = peaks[i + j][IDX_TIME_J]
                        t_delta = t2 - t1
                        if t_delta >= MIN_HASH_TIME_DELTA and t_delta <= MAX_HASH_TIME_DELTA:
                            h = hashlib.sha1(
                                "%s|%s|%s" % (str(freq1), str(freq2), str(t_delta)))
                            yield (h.hexdigest()[0:FINGERPRINT_REDUCTION], t1)
        def fingerprint(channel_samples, Fs=DEFAULT_FS,
                        wsize=DEFAULT_WINDOW_SIZE,
                        wratio=DEFAULT_OVERLAP_RATIO,
                        fan_value=DEFAULT_FAN_VALUE,
                        amp_min=DEFAULT_AMP_MIN):
            # FFT the signal and extract frequency components
            arr2D = mlab.specgram(
                    channel_samples,
                    NFFT=wsize,
                    Fs=Fs,
                    window=mlab.window_hanning,
                    noverlap=int(wsize * wratio))[0]
            # apply log transform since specgram() returns linear array
            arr2D = 10 * np.log10(arr2D)
            arr2D[arr2D == -np.inf] = 0  # replace infs with zeros
            # find local maxima
            local_maxima = get_2D_peaks(arr2D, plot=False, amp_min=amp_min)
            # return hashes
            return generate_hashes(local_maxima, fan_value=fan_value)
        
        buf = audio_bit.split(',')
        buf = map(int, buf)
        buf = array('b',buf)
        buf = buf.tostring()
        #buf = array(‘b‘, map(int, audio_bit.split(‘,‘))).tostring()
        aud_arr = buf
        input_file = NamedTemporaryFile(mode='wb', delete=False)
        input_file.write(buf)
        input_file.flush()
        tempfilename = open(input_file.name,mode="rb")
        aud_arr = tempfilename.read()
        plpy.info("after reading from temp file");
        plpy.info("before decoding");
        audiobitstream=aud_arr
        #channels, Fs = read(audiobitstream, limit)
        channels, Fs = read_frombuffer_so(buf)
        plpy.info("after decoding");
        result = set()
        channel_amount = len(channels)
        for channeln, channel in enumerate(channels):
            # TODO: Remove prints or change them into optional logging.
            print("Fingerprinting channel %d/%d " % (channeln + 1,
                                                           channel_amount
                                                           ))
            hashes = fingerprint(channel, Fs=Fs)
            print("Finished channel %d/%d " % (channeln + 1, channel_amount
                                                     ))
            result |= set(hashes)
        plpy.info("after fingerprinting")
        import pandas as pd
        song_names = []
        time_offsets = np.zeros(len(result),dtype=np.float)
        fing = []
        i = 0
        plpy.info(len(time_offsets))
        for hash_v in result:
            song_names.append(song_name)
            fing.append(hash_v[0])
            time_offsets[i] = hash_v[1]
            yield(time_offsets[i],fing[i])
            i=i+1
        #return(pd.DataFrame(zip(time_offsets,fing)))
    $$ language plpythonu;
"""
psql.execute(sql,conn)
conn.commit()

In [154]:
sql = """drop function if exists audiofp.fing_extract_ret_test() cascade;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create or replace function audiofp.fing_extract_ret_test() 
    returns setof fingerprint_result
    as 
    $$
        import pandas as pd
        import numpy as np
        
        time_offset = np.zeros(4,dtype=np.float)
        fing = []
       
        plpy.info(len(time_offset))
        for i in range(4):
            fing.append(i)
            time_offset[i] = i
            yield(time_offset[i],fing[i])
            
        #return(pd.DataFrame(zip(time_offset,fing),columns=list('a','b')))
    $$ language plpythonu;
"""
psql.execute(sql,conn)
conn.commit()

In [155]:
sql = """select audiofp.fing_extract_ret_test();"""
psql.read_sql(sql,conn)

,fing_extract_ret_test
0,"(0,0)"
1,"(1,1)"
2,"(2,2)"
3,"(3,3)"


In [20]:
# First let us compute the SIFT features using the Python binding for the OpenCV 
# library locally in this notebook to ensure our images have been stored properly
import hexdump
sql = """
        SELECT
            aud_name,
            audio
        FROM
            audiofp.training_audio
        WHERE aud_name like '%/user/radhar5/audio/Aha.mp3%'
        LIMIT 1;        
"""

df = psql.read_sql(sql,conn)

imgname = np.array(df.loc[0]['aud_name'])

aud = df.loc[0]['audio']
print aud[0:10]
buf = aud.split(',')
buf = map(int, buf)
buf = array('b',buf)
buf = buf.tostring()
print(type(buf))
print(hexdump.dump(bytearray(buf[0:10]),sep=':'))
aud_arr = np.asarray(bytearray(buf), dtype=np.uint8)
#print(aud_arr[0:10])
print(len(aud_arr))
AudioSegment.converter = "/Applications/ffmpeg"
extract_fing_from_buffer(aud_arr,limit=None,song_name='test_name')

73, 68, 51
<type 'str'>
49:44:33:04:00:00:00:00:01:0C
3783724
read audio bitstream
after reading audio bitstream
44100
2
44100
Fingerprinting channel 1/2 
Finished channel 1/2 
Fingerprinting channel 2/2 
Finished channel 2/2 


,0,1,2
0,test_name,874,20326c751c77c3b38257
1,test_name,4706,25e26a768c88d902b981
2,test_name,1268,9e139b5ffe793ca86750
3,test_name,1624,08a2479ca87c5e71bf39
4,test_name,2043,b809b601260823946dbd
5,test_name,873,84b702a51a636913b45b
6,test_name,3649,d1b74ba6155bb3c0196e
7,test_name,1864,2c85b2f907dc98004a03
8,test_name,393,9c1a2213113c606aa4d5
9,test_name,4924,51e40e5fadd2c1ebc1b1


In [178]:

sql = """
   select
         aud_name,
         (fing_descriptors).*
         
  from
  ( select
       aud_name,
       audiofp.fing_extract_plpy(audio) as fing_descriptors
   from
        audiofp.training_audio
    where
        aud_name like '%Beyonce-_Halo.mp3vlc_cmd.mp3%'
    limit 10
  ) as foo
"""

df = psql.read_sql(sql,conn)


In [102]:
print(df)

                      aud_name  time_offset              fing_val
0  /user/radhar5/audio/ash.mp3         2240  e926d448d2ffdeac8f2f
1  /user/radhar5/audio/ash.mp3         2464  ceecad194cca1ecbacb3
2  /user/radhar5/audio/ash.mp3         1623  2f6c78c1c47feadeca2b
3  /user/radhar5/audio/ash.mp3         2959  e7ca478b8fc5db0a7cbd
4  /user/radhar5/audio/ash.mp3         1717  98be61e4b8690af83d39
5  /user/radhar5/audio/ash.mp3           39  8732a29a0561702bdb3d
6  /user/radhar5/audio/ash.mp3         1663  ebdf21b88977693124ce
7  /user/radhar5/audio/ash.mp3         2588  cd23a4bea728ba10ea2b
8  /user/radhar5/audio/ash.mp3         1493  259a75363a145d50d31a
9  /user/radhar5/audio/ash.mp3         3291  292ea0f9a10d823c062e


In [196]:
sql = """drop table if exists audiofp.training_fing_descriptors_vlcmp3_1;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table audiofp.training_fing_descriptors_vlcmp3_1 as (
       select
             aud_name,
             (fing_descriptors).*
      from
      ( select
           aud_name,
           audiofp.fing_extract_plpy(audio) as fing_descriptors
       from
            audiofp.training_audio
       where aud_name NOT like '%Beyonce-_Halo.mp3vlc_cmd.mp3%'
	      and aud_name NOT like '%Black_Eyed_Peas-_Boom_Boom_Pow%'
	      and aud_name NOT like '%Black_Eyed_Peas-_I_Gotta_Feeling%'
	      and aud_name NOT like '%Green_Day-_21_Guns%'
	      and aud_name NOT like '%Katy_Perry-_Hot_N_Cold%'
	      and aud_name NOT like '%Kings_Of_Leon-_Sex_On_Fire%'
	      and aud_name NOT like '%Lily_Allen-_The_Fear%'
	      and aud_name NOT like '%Linkin_Park-_New_Divide%'
	      and aud_name NOT like '%Pitbull-_I_Know_You_Want_Me%'
      ) as foo
    ) distributed randomly;
"""
psql.execute(sql,conn)
conn.commit()

In [207]:
sql = """drop table if exists audiofp.training_fing_descriptors_vlcmp3;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table audiofp.training_fing_descriptors_vlcmp31 as (
       select
             aud_name,
             (fing_descriptors).*
      from
      ( select
           aud_name,
           audiofp.fing_extract_plpy(audio) as fing_descriptors
       from
            audiofp.training_audio
       
      ) as foo
    ) distributed randomly;
"""
psql.execute(sql,conn)
conn.commit()

In [208]:
sql="""select count(*) from audiofp.training_fing_descriptors_vlcmp3"""
psql.read_sql(sql,conn)

,count
0,7024286


In [197]:
sql="""select count(*) from audiofp.training_fing_descriptors_vlcmp3_1"""
psql.read_sql(sql,conn)

,count
0,5696564


In [198]:
sql = """insert into audiofp.training_fing_descriptors_vlcmp3
(select
      *
from
    audiofp.training_fing_descriptors
where
    aud_name like '%ash.mp3%'
)
""" 
psql.read_sql(sql,conn)

TypeError: 'NoneType' object is not iterable

In [160]:
sql = """drop external table if exists audiofp.testing_audio_ext;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create external table audiofp.testing_audio_ext (
        aud_name varchar,
        audio varchar
    )
    LOCATION('gpfdist://mdw:8089/audio_seq_file_testrec.csv')
    FORMAT 'TEXT' (DELIMITER AS E'\t');
"""
psql.execute(sql,conn)
conn.commit()

In [161]:
sql = """select count(*) from audiofp.testing_audio_ext"""
psql.read_sql(sql,conn)

,count
0,5


In [162]:
sql = """drop table if exists audiofp.testing_audio;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table audiofp.testing_audio AS
        (select * from audiofp.testing_audio_ext)
    distributed by (aud_name)
"""
psql.execute(sql,conn)
conn.commit()

In [163]:
sql= """select aud_name from audiofp.testing_audio order by 1"""
psql.read_sql(sql,conn)


,aud_name
0,/user/radhar5/audio/Aha1.mp3
1,/user/radhar5/audio/Aha2.mp3
2,/user/radhar5/audio/Ash1.mp3
3,/user/radhar5/audio/Ash2.mp3
4,/user/radhar5/audio/Ash3.mp3


In [164]:
sql = """drop table if exists audiofp.testing_fing_descriptors;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table audiofp.testing_fing_descriptors as (
       select
             aud_name,
             (fing_descriptors).*
      from
      ( select
           aud_name,
           audiofp.fing_extract_plpy(audio) as fing_descriptors
       from
            audiofp.testing_audio
      ) as foo
    ) distributed randomly;
"""
psql.execute(sql,conn)
conn.commit()

In [123]:
sql = """drop external table if exists audiofp.testing_audio_ext1;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create external table audiofp.testing_audio_ext1 (
        aud_name varchar,
        audio varchar
    )
    LOCATION('gpfdist://localhost:8089/audio_seq_file_ashcut.csv')
    FORMAT 'TEXT' (DELIMITER AS E'\t');
"""
psql.execute(sql,conn)
conn.commit()

In [124]:
sql = """select count(*) from audiofp.testing_audio_ext1"""
psql.read_sql(sql,conn)

,count
0,2


In [125]:
sql = """drop table if exists audiofp.testing_audio1;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table audiofp.testing_audio1 AS
        (select * from audiofp.testing_audio_ext1)
    distributed by (aud_name)
"""
psql.execute(sql,conn)
conn.commit()

In [126]:
sql= """select aud_name from audiofp.testing_audio1 order by 1"""
psql.read_sql(sql,conn)


,aud_name
0,/user/radhar5/audio/ashcut1.mp3
1,/user/radhar5/audio/ashcut2.mp3


In [116]:
sql = """drop table if exists audiofp.testing_fing_descriptors1;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table audiofp.testing_fing_descriptors1 as (
       select
             aud_name,
             (fing_descriptors).*
      from
      ( select
           aud_name,
           audiofp.fing_extract_plpy(audio) as fing_descriptors
       from
            audiofp.testing_audio1
      ) as foo
    ) distributed randomly;
"""
psql.execute(sql,conn)
conn.commit()

DatabaseError: Execution failed on sql '
    create table audiofp.testing_fing_descriptors1 as (
       select
             aud_name,
             (fing_descriptors).*
      from
      ( select
           aud_name,
           audiofp.fing_extract_plpy(audio) as fing_descriptors
       from
            audiofp.testing_audio1
      ) as foo
    ) distributed randomly;
': Unexpected internal error: Segment process received signal SIGSEGV (postgres.c:3442)  (seg1 slice1 localhost.localdomain:40001 pid=56516) (cdbdisp.c:1475)
DETAIL:  
	 (interrupt holdoff count 0, critical section count 0)
	 PL/Python function "fing_extract_plpy"


In [117]:
sql = """drop external table if exists audiofp.testing_audio_ext1;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create external table audiofp.testing_audio_ext1 (
        aud_name varchar,
        audio varchar
    )
    LOCATION('gpfdist://localhost:8089/audio_seq_file_ashcutm4a.csv')
    FORMAT 'TEXT' (DELIMITER AS E'\t');
"""
psql.execute(sql,conn)
conn.commit()

In [118]:
sql = """select count(*) from audiofp.testing_audio_ext1"""
psql.read_sql(sql,conn)

,count
0,2


In [119]:
sql = """drop table if exists audiofp.testing_audio1;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table audiofp.testing_audio1 AS
        (select * from audiofp.testing_audio_ext1)
    distributed by (aud_name)
"""
psql.execute(sql,conn)
conn.commit()

In [120]:
sql= """select aud_name from audiofp.testing_audio1 order by 1"""
psql.read_sql(sql,conn)

,aud_name
0,/user/radhar5/audio/ashcut1.m4a
1,/user/radhar5/audio/ashcut2.m4a


In [122]:
sql = """drop table if exists audiofp.testing_fing_descriptors1;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table audiofp.testing_fing_descriptors1 as (
       select
             aud_name,
             (fing_descriptors).*
      from
      ( select
           aud_name,
           audiofp.fing_extract_plpy(audio) as fing_descriptors
       from
            audiofp.testing_audio1
      ) as foo
    ) distributed randomly;
"""
psql.execute(sql,conn)
conn.commit()

DatabaseError: Execution failed on sql '
    create table audiofp.testing_fing_descriptors1 as (
       select
             aud_name,
             (fing_descriptors).*
      from
      ( select
           aud_name,
           audiofp.fing_extract_plpy(audio) as fing_descriptors
       from
            audiofp.testing_audio1
      ) as foo
    ) distributed randomly;
': Error on receive from seg0 slice1 localhost.localdomain:40000 pid=58258: server closed the connection unexpectedly
DETAIL:  
	This probably means the server terminated abnormally
	before or while processing the request.


In [134]:
sql="""select count(*) from audiofp.training_fing_descriptors"""
psql.read_sql(sql,conn)

,count
0,669243


In [203]:
sql = """select db_aud,time_diff,count(*)
from
(
select
      t1.*,
      t2.aud_name db_aud,
      t2.time_offset db_off,
      t2.time_offset-t1.time_offset time_diff
from
(
(select
      * 
from
    audiofp.testing_fing_descriptors
where
    aud_name like '%/user/radhar5/audio/Ash3.mp3%'
)t1
inner join
(
select
      * 
from
    audiofp.training_fing_descriptors_vlcmp3
)t2
on(t1.fing_val=t2.fing_val)
)
)as foo group by 1,2 order by 3 desc"""
psql.read_sql(sql,conn)

,db_aud,time_diff,count
0,/user/radhar5/audio/A-Ha-_Foot_Of_The_Mountain...,893,106
1,/user/radhar5/audio/A-Ha-_Foot_Of_The_Mountain...,2483,13
2,/user/radhar5/audio/ash.mp3,573,4
3,/user/radhar5/audio/Kid_Rock-_All_Summer_Long....,5406,3
4,/user/radhar5/audio/David_Guetta-_Sexy_Bitch.m...,121,3
5,/user/radhar5/audio/Lady_GaGa-_Poker_Face.mp3v...,2226,3
6,/user/radhar5/audio/Milow-_Ayo_Technology.mp3v...,2584,3
7,/user/radhar5/audio/Culcha_Candela-_Schone_Neu...,4218,3
8,/user/radhar5/audio/A-Ha-_Foot_Of_The_Mountain...,1945,3
9,/user/radhar5/audio/Cassandra_Steen-_Stadt.mp3...,575,3


In [246]:
# We will first create the UDF
sql = """drop function if exists audiofp.extract_fing_from_soundcloud_url(sound_cloud_url varchar) cascade;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create or replace function audiofp.extract_fing_from_soundcloud_url(sound_cloud_url varchar) 
    returns setof fingerprint_result
    as 
    $$
        import plpy
        plpy.info('function invoked')
        import os
        import fnmatch
        import numpy as np
        #import mlab
        #from pydub import AudioSegment
        #from pydub.utils import audioop
        #import wavio
        from hashlib import sha1
        #import wave as _wave
        import numpy as _np
        from array import array
        from tempfile import TemporaryFile, NamedTemporaryFile

        import numpy as np
        import matplotlib.mlab as mlab
        #import matplotlib.pyplot as plt
        from scipy.ndimage.filters import maximum_filter
        from scipy.ndimage.morphology import (generate_binary_structure,
                                              iterate_structure, binary_erosion)
        import hashlib
        from operator import itemgetter
        import wave as _wave

        song_name = None
        limit = None
        
        IDX_FREQ_I = 0
        IDX_TIME_J = 1

       
        DEFAULT_FS = 44100

        
        DEFAULT_WINDOW_SIZE = 4096

       
        DEFAULT_OVERLAP_RATIO = 0.5

        
        DEFAULT_FAN_VALUE = 15

        
        DEFAULT_AMP_MIN = 10

       
        PEAK_NEIGHBORHOOD_SIZE = 20

        
        MIN_HASH_TIME_DELTA = 0
        MAX_HASH_TIME_DELTA = 200

       
        PEAK_SORT = True

        FINGERPRINT_REDUCTION = 20

        def _wav2array(nchannels, sampwidth, data):
            num_samples, remainder = divmod(len(data), sampwidth * nchannels)
            if remainder > 0:
                raise ValueError('The length of data is not a multiple of '
                                 'sampwidth * num_channels.')
            if sampwidth > 4:
                raise ValueError("sampwidth must not be greater than 4.")
            if sampwidth == 3:
                a = _np.empty((num_samples, nchannels, 4), dtype=_np.uint8)
                raw_bytes = _np.fromstring(data, dtype=_np.uint8)
                a[:, :, :sampwidth] = raw_bytes.reshape(-1, nchannels, sampwidth)
                a[:, :, sampwidth:] = (a[:, :, sampwidth - 1:sampwidth] >> 7) * 255
                result = a.view('<i4').reshape(a.shape[:-1])
            else:
                # 8 bit samples are stored as unsigned ints; others as signed ints.
                dt_char = 'u' if sampwidth == 1 else 'i'
                a = _np.fromstring(data, dtype='<%s%d' % (dt_char, sampwidth))
                result = a.reshape(-1, nchannels)
            return result
        def readwav(file):
            wav = _wave.open(file)
            rate = wav.getframerate()
            nchannels = wav.getnchannels()
            sampwidth = wav.getsampwidth()
            nframes = wav.getnframes()
            data = wav.readframes(nframes)
            wav.close()
            array = _wav2array(nchannels, sampwidth, data)
            return rate, sampwidth, array
        def read_from_wavfile(filename):
            fs, _, audiofile = readwav(filename)
            audiofile = audiofile.T
            audiofile = audiofile.astype(np.int16)
            channels = []
            for chn in audiofile:
                channels.append(chn)
            return channels, fs
        def get_2D_peaks(arr2D, plot=False, amp_min=DEFAULT_AMP_MIN):
            # http://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.morphology.iterate_structure.html#scipy.ndimage.morphology.iterate_structure
            struct = generate_binary_structure(2, 1)
            neighborhood = iterate_structure(struct, PEAK_NEIGHBORHOOD_SIZE)
            # find local maxima using our fliter shape
            local_max = maximum_filter(arr2D, footprint=neighborhood) == arr2D
            background = (arr2D == 0)
            eroded_background = binary_erosion(background, structure=neighborhood,
                                               border_value=1)
            # Boolean mask of arr2D with True at peaks
            detected_peaks = local_max - eroded_background
            # extract peaks
            amps = arr2D[detected_peaks]
            j, i = np.where(detected_peaks)
            # filter peaks
            amps = amps.flatten()
            peaks = zip(i, j, amps)
            peaks_filtered = [x for x in peaks if x[2] > amp_min]  # freq, time, amp
            # get indices for frequency and time
            frequency_idx = [x[1] for x in peaks_filtered]
            time_idx = [x[0] for x in peaks_filtered]
            if plot:
                # scatter of the peaks
                fig, ax = plt.subplots()
                ax.imshow(arr2D)
                ax.scatter(time_idx, frequency_idx)
                ax.set_xlabel('Time')
                ax.set_ylabel('Frequency')
                ax.set_title("Spectrogram")
                plt.gca().invert_yaxis()
                plt.show()
            return zip(frequency_idx, time_idx)
        def generate_hashes(peaks, fan_value=DEFAULT_FAN_VALUE):
            if PEAK_SORT:
                peaks.sort(key=itemgetter(1))
            for i in range(len(peaks)):
                for j in range(1, fan_value):
                    if (i + j) < len(peaks):
                        freq1 = peaks[i][IDX_FREQ_I]
                        freq2 = peaks[i + j][IDX_FREQ_I]
                        t1 = peaks[i][IDX_TIME_J]
                        t2 = peaks[i + j][IDX_TIME_J]
                        t_delta = t2 - t1
                        if t_delta >= MIN_HASH_TIME_DELTA and t_delta <= MAX_HASH_TIME_DELTA:
                            h = hashlib.sha1(
                                "%s|%s|%s" % (str(freq1), str(freq2), str(t_delta)))
                            yield (h.hexdigest()[0:FINGERPRINT_REDUCTION], t1)
        def fingerprint(channel_samples, Fs=DEFAULT_FS,
                        wsize=DEFAULT_WINDOW_SIZE,
                        wratio=DEFAULT_OVERLAP_RATIO,
                        fan_value=DEFAULT_FAN_VALUE,
                        amp_min=DEFAULT_AMP_MIN):
            # FFT the signal and extract frequency components
            arr2D = mlab.specgram(
                    channel_samples,
                    NFFT=wsize,
                    Fs=Fs,
                    window=mlab.window_hanning,
                    noverlap=int(wsize * wratio))[0]
            # apply log transform since specgram() returns linear array
            arr2D = 10 * np.log10(arr2D)
            arr2D[arr2D == -np.inf] = 0  # replace infs with zeros
            # find local maxima
            local_maxima = get_2D_peaks(arr2D, plot=False, amp_min=amp_min)
            # return hashes
            return generate_hashes(local_maxima, fan_value=fan_value)
        import soundcloud
        plpy.info("bef soundcloud client")
        if 'client' not in GD:
            plpy.info("inside GD client")
            client = soundcloud.Client(client_id='a4180fa6d976ef389b7685c2779535cf',client_secret='55a848f7ec1660f36659c2459f8e2451',username='regunathanr@gmail.com',password='shivani8')
            GD['client'] = client
        else:
            plpy.info("getting client from GD")
            client = GD['client']
        plpy.info(client.get('/me').username)
        track = client.get('/resolve', url=sound_cloud_url)
        download_url = client.get(track.download_url, allow_redirects=False)
        plpy.info(download_url.url)
        import urllib2
        wavfile_sndcld = urllib2.urlopen(download_url.url)
        input_file = NamedTemporaryFile(mode='wb', delete=False)
        input_file.write(wavfile_sndcld.read())
        input_file.flush()
        channels,Fs = read_from_wavfile(input_file.name)
        plpy.info("before fing")
        result = set()
        channel_amount = len(channels)
        for channeln, channel in enumerate(channels):
            # TODO: Remove prints or change them into optional logging.
            print("Fingerprinting channel %d/%d " % (channeln + 1,
                                                           channel_amount
                                                           ))
            hashes = fingerprint(channel, Fs=Fs)
            print("Finished channel %d/%d " % (channeln + 1, channel_amount
                                                     ))
            result |= set(hashes)
        plpy.info("after fingerprinting")
        import pandas as pd
        song_names = []
        time_offsets = np.zeros(len(result),dtype=np.float)
        fing = []
        i = 0
        plpy.info(len(time_offsets))
        for hash_v in result:
            song_names.append(song_name)
            fing.append(hash_v[0])
            time_offsets[i] = hash_v[1]
            yield(time_offsets[i],fing[i])
            i=i+1
        #return(pd.DataFrame(zip(time_offsets,fing)))
    $$ language plpythonu;
"""
psql.execute(sql,conn)
conn.commit()

In [231]:
sql = """select audiofp.extract_fing_from_soundcloud_url('https://soundcloud.com/user-558890072-861956/beyo_halo_2')
"""
psql.read_sql(sql,conn)

,extract_fing_from_soundcloud_url
0,"(23,35d26ac01adb5da3a227)"
1,"(88,8e90a09d2bbeed2156ca)"
2,"(67,ec823377102db3fe89df)"
3,"(23,43793d24f99810cd38c5)"
4,"(88,93a01dec21397046f325)"
5,"(152,1465ae46557029bf9c43)"
6,"(23,e3917bc7808589e838a1)"
7,"(67,0b7c73d1779e64717c11)"
8,"(0,7e639823db2477535893)"
9,"(24,2bd377afa3e54a19cc16)"


In [233]:
sql = """drop table if exists audiofp.testing_fing_descriptors_sndcld;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table audiofp.testing_fing_descriptors_sndcld 
    (aud_name text,
    time_offset float, fing_val text) distributed randomly"""
psql.execute(sql,conn)
conn.commit()

sql = """insert into audiofp.testing_fing_descriptors_sndcld (
       select
             'test'::text as aud_name,
             (fing_descriptors).*
      from
      ( select
           
            audiofp.extract_fing_from_soundcloud_url('https://soundcloud.com/user-558890072-861956/beyo_halo_2') as fing_descriptors
       
      ) as foo
    ) 
    
   
"""
psql.execute(sql,conn)
conn.commit()

DatabaseError: Execution failed on sql 'insert into audiofp.testing_fing_descriptors_sndcld (
       select
             'test'::text as aud_name,
             (fing_descriptors).*
      from
      ( select
           
            audiofp.extract_fing_from_soundcloud_url('https://soundcloud.com/user-558890072-861956/beyo_halo_2') as fing_descriptors
       
      ) as foo
    ) 
': function "extract_fing_from_soundcloud_url" error fetching next item from iterator (plpython.c:4648)  (seg0 slice1 sdw10:40000 pid=25129) (cdbdisp.c:1476)
DETAIL:  
	 requests.exceptions.ConnectionError: HTTPSConnectionPool(host='api.soundcloud.com', port=443): Max retries exceeded with url: /oauth2/token (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0xe866cd0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))
	 Traceback (most recent call last):
  PL/Python function "extract_fing_from_soundcloud_url", line 43, 

In [ ]:
sql = """select db_aud,time_diff,count(*)
from
(
select
      t1.*,
      t2.aud_name db_aud,
      t2.time_offset db_off,
      t2.time_offset-t1.time_offset time_diff
from
(
    (     select
                 'test'::text as aud_name,
                 (fing_descriptors).*
          from
          ( select

                audiofp.extract_fing_from_soundcloud_url('https://soundcloud.com/user-558890072-861956/beyo_halo_2') as fing_descriptors

          ) as foo
    )t1
inner join
    (
    select
          * 
    from
        audiofp.training_fing_descriptors_vlcmp3
    )t2
on(t1.fing_val=t2.fing_val)
)
)as foo group by 1,2 order by 3 desc"""
psql.read_sql(sql,conn)

In [259]:
sql= """copy (select
             'test'::text as aud_name,
             (fing_descriptors).*
      from
      ( select
           
            audiofp.extract_fing_from_soundcloud_url('https://soundcloud.com/user-558890072-861956/beyo_halo_2') as fing_descriptors
       
      ) as foo) to '/home/gpadmin/test_output1.csv' with CSV"""

psql.execute(sql,conn)

<cursor object at 0x113be9bf0; closed: 0>

In [254]:
sql = """drop table if exists audiofp.testing_fing_descriptors_sndcld;"""
psql.execute(sql,conn)
conn.commit()

sql = """
    create table audiofp.testing_fing_descriptors_sndcld 
    (aud_name text,
    time_offset float, fing_val text) distributed randomly"""
psql.execute(sql,conn)
conn.commit()

sql = """copy audiofp.testing_fing_descriptors_sndcld from '/home/gpadmin/test_output1.csv' DELIMITER ',' CSV"""
psql.execute(sql,conn)
conn.commit()

In [253]:
sql = """drop table if exists audiofp.testing_fing_descriptors_sndcld;
create table audiofp.testing_fing_descriptors_sndcld 
    (aud_name text,
    time_offset float, fing_val text) distributed randomly;
    copy audiofp.testing_fing_descriptors_sndcld from '/home/gpadmin/test_output1.csv' DELIMITER ',' CSV;"""

psql.read_sql(sql,conn)

TypeError: 'NoneType' object is not iterable

In [256]:
sql = """select db_aud,time_diff,count(*)
from
(
select
      t1.*,
      t2.aud_name db_aud,
      t2.time_offset db_off,
      (t2.time_offset-t1.time_offset)*(2048.0/44100) time_diff
from
(
(select
      * 
from
    audiofp.testing_fing_descriptors_sndcld

)t1
inner join
(
select
      * 
from
    audiofp.training_fing_descriptors_vlcmp3
)t2
on(t1.fing_val=t2.fing_val)
)
)as foo group by 1,2 order by 3 desc"""
psql.read_sql(sql,conn)

,db_aud,time_diff,count
0,/user/radhar5/audio/Beyonce-_Halo.mp3vlc_cmd.mp3,61.811519,706
1,/user/radhar5/audio/Beyonce-_Halo.mp3vlc_cmd.mp3,133.793379,54
2,/user/radhar5/audio/Beyonce-_Halo.mp3vlc_cmd.mp3,208.793832,23
3,/user/radhar5/audio/Beyonce-_Halo.mp3vlc_cmd.mp3,57.306848,14
4,/user/radhar5/audio/Beyonce-_Halo.mp3vlc_cmd.mp3,27.306667,11
5,/user/radhar5/audio/Beyonce-_Halo.mp3vlc_cmd.mp3,22.801995,10
6,/user/radhar5/audio/Beyonce-_Halo.mp3vlc_cmd.mp3,28.792744,9
7,/user/radhar5/audio/Beyonce-_Halo.mp3vlc_cmd.mp3,112.802540,8
8,/user/radhar5/audio/Beyonce-_Halo.mp3vlc_cmd.mp3,69.288345,6
9,/user/radhar5/audio/Beyonce-_Halo.mp3vlc_cmd.mp3,87.307029,6
